In [1]:
pwd

'/home/wsuser/work'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:

!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=724803080c7dd187ddde0579e8bd0ac9f5c242f2da03abe222f46e5f92415bba
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/4b/a5/2d/4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZLCeTQpk4Ciujmb8y3B8B2x3euF2U1MrvGyTcHv0I37i',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'parkinson-donotdelete-pr-jze28w3uemad1x'
object_key = 'dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)


In [7]:
pwd


'/home/wsuser/work'

In [8]:
def quantify_image(image):
    features = feature.hog(image, 
                           orientations=9, 
                           pixels_per_cell=(5,5), 
                           cells_per_block=(2,2), 
                           transform_sqrt=True, 
                           block_norm="L1")
    return features

In [9]:
def load_split(path):
    path_images = list(paths.list_images(path))
    data=[]
    labels=[]

    for path_image in path_images:
        label = path_image.split(os.path.sep)[-2]
        image = cv2.imread(path_image)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200,200))
        image = cv2.threshold(image,0,225,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

        features = quantify_image(image)
        data.append(features)
        labels.append(label)

    return (np.array(data), np.array(labels))

In [10]:
path_training_data = r"/home/wsuser/work/dataset/spiral/training"
path_testing_data = r"/home/wsuser/work/dataset/spiral/testing"

In [11]:
(x_train, y_train) = load_split(path_training_data)
(x_test, y_test) = load_split(path_testing_data)

In [12]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
print(x_train.shape, y_train.shape)
# 0:healthy,1:Parkinson

(72, 54756) (72,)


In [13]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

RandomForestClassifier()

In [14]:
testingPaths = list(paths.list_images(path_testing_data))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False) 
images = []

In [15]:
for i in idxs:
    # loading the testing image, clone it, and resize it 
    image = cv2.imread(testingPaths[i]) 
    output = image. copy() 
    output = cv2. resize(output, (128, 128))
    
    # pre-processing the image 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU) [1]
    
    # quantify the image and make predictions based on the extracted 
    # features using the last trained Random Forest 
    features = quantify_image(image) 
    preds = model.predict([features])

    label = label_encoder.inverse_transform(preds)[0]

    # draw the colored class label on the output image and add it to the set of output images 
    if label == "healthy":
        color = (0, 255, 0)  
    else:
        color = (0, 0, 255) 
    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) 
    images.append(output)

In [56]:
predictions = model.predict(x_test)    # predictions on the testing data 

cm = confusion_matrix(y_test, predictions).flatten ()    # computing the confusion matrix
print(cm) 
(tn, fp, fn, tp) = cm 

accuracy = (tp + tn) / float(cm.sum())     # computing the accuracy
print(accuracy)

[14  1  5 10]
0.8


In [57]:
pickle.dump(model,open('parkinson.pkl', 'wb'))

In [58]:
!tar -zcvf parkinsonsmodel.tgz parkinson.pkl

parkinson.pkl


In [59]:
ls

dataset/                                   parkinson.pkl
Deployed_Parkinson_Detection_Model.tar.gz  parkinsonsmodel.tgz


In [60]:
path_training_data = r"dataset/wave/training"
path_testing_data = r"dataset/wave/testing"

In [61]:
(x_train, y_train) = load_split(path_training_data)
(x_test, y_test) = load_split(path_testing_data)

In [62]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
print(x_train.shape, y_train.shape)
# 0:healthy,1:Parkinson

(72, 54756) (72,)


In [64]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

RandomForestClassifier()

In [65]:
testingPaths = list(paths.list_images(path_testing_data))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False) 
images = []

In [66]:
for i in idxs:
    # loading the testing image, clone it, and resize it 
    image = cv2.imread(testingPaths[i]) 
    output = image. copy() 
    output = cv2. resize(output, (128, 128))
    
    # pre-processing the image 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU) [1]
    
    # quantify the image and make predictions based on the extracted 
    # features using the last trained Random Forest 
    features = quantify_image(image) 
    preds = model.predict([features])

    label = label_encoder.inverse_transform(preds)[0]

    # draw the colored class label on the output image and add it to the set of output images 
    if label == "healthy":
        color = (0, 255, 0)  
    else:
        color = (0, 0, 255) 
    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) 
    images.append(output)

In [67]:
predictions = model.predict(x_test)    # predictions on the testing data 

cm = confusion_matrix(y_test, predictions).flatten ()    # computing the confusion matrix
print(cm) 
(tn, fp, fn, tp) = cm 

accuracy = (tp + tn) / float(cm.sum())     # computing the accuracy
print(accuracy)

[9 6 6 9]
0.6


In [68]:
pickle.dump(model,open('parkinson_w.pkl', 'wb'))

In [69]:
!tar -zcvf parkinsons-detection-model_s.tgz parkinson_w.pkl

parkinson_w.pkl


In [70]:
ls

dataset/                                   parkinsons-detection-model_s.tgz
Deployed_Parkinson_Detection_Model.tar.gz  parkinsonsmodel.tgz
parkinson.pkl                              parkinson_w.pkl


In [71]:
!pip install watson-machine-learning-client --upgrade


In [72]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url" : "https://us-south.ml.cloud.ibm.com",
                    "apikey" : "H_iuH6LqaEKth6oWkVUVoTLlbvX012wRF5gZpR82oYRz"
                   }
client = APIClient(wml_credentials)

In [73]:
client=APIClient(wml_credentials)
client

In [74]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------------  ------------------------
ID                                    NAME             CREATED
1768bd20-dae5-491c-9ec0-23ec8a326b35  parkinsondetect  2022-11-22T13:59:55.949Z
------------------------------------  ---------------  ------------------------


In [75]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    print(space)
    return (next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [76]:
space_uid = guid_from_space_name(client,'parkinsondetect')
print('Space UID = ' + space_uid)

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/aa6dd997b39e4214bc986a44dc329421:c6904334-67fb-439e-acb2-2250b9323ee1::', 'guid': 'c6904334-67fb-439e-acb2-2250b9323ee1', 'name': 'Watson Machine Learning-mo', 'type': 'machine_learning'}], 'description': '', 'name': 'parkinsondetect', 'scope': {'bss_account_id': 'aa6dd997b39e4214bc986a44dc329421'}, 'stage': {'production': False}, 'status': {'state': 'active'}, 'storage': {'properties': {'bucket_name': '403c7ccb-35e7-446e-a741-dc8b2746bec5', 'bucket_region': 'us-south', 'credentials': {'admin': {'access_key_id': '470a4952fad7461e9a69f81186424bf0', 'api_key': '4yxvBaIe-6PRgOkM0hb6rjuK-UzOMruopKmGPd0YcgQ_', 'secret_access_key': '66919f627e8e096aa60886f88b2e040e26f3d5d126943584', 'service_id': 'ServiceId-3e45e4f8-3d60-401c-b84b-dfa3b250d546'}, 'editor': {'access_key_id': 'e11ecd2940344708b3d97520ff4ee026', 'api_key': 'tmnOrbxQGO23yHIVrjCDw7XnCKmh2JPZyTFp-tq_FJfX', 'resource_key_crn': 'crn:v1:bluemix:pub

In [77]:
client.set.default_space(space_uid)

'SUCCESS'

In [78]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [79]:
import sklearn
sklearn.__version__

'1.0.2'

In [80]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [81]:
model_details = client.repository.store_model(model='parkinsonsmodel.tgz',meta_props = {
client.repository.ModelMetaNames.NAME: "parkinsondetect", 
client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0", 
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})
model_id = client.repository.get_model_id(model_details)


In [82]:
model_id

'8dc6d7e1-8141-4767-ab34-67a05609d7bb'

In [84]:
client.repository.download(model_id,'Deployed_Parkinson_Model.tar.gz')

Successfully saved model content to file: 'Deployed_Parkinson_Model.tar.gz'


'/home/wsuser/work/Deployed_Parkinson_Model.tar.gz'

In [85]:
#Deployment in props

deployment_props = {
    client.deployments.ConfigurationMetaNames.NAME : 'Detecting Parkinson Disease Using Machine Learning-Deployment',
    client.deployments.ConfigurationMetaNames.ONLINE : {}
}

In [86]:
deployment = client.deployments.create(
    artifact_uid = model_id,
    meta_props = deployment_props
)



#######################################################################################

Synchronous deployment creation for uid: '8dc6d7e1-8141-4767-ab34-67a05609d7bb' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0f1d1533-f688-4e7d-9bd3-69b49744eed6'
------------------------------------------------------------------------------------------------


